In [ ]:
import os
os.chdir(r'\\share.ipp-hgw.mpg.de\documents\pih\Documents\git\QSB_Bolometry\libprad')

import importlib
import archivedb
import numpy as np
import matplotlib.pyplot as p

import plot_funcs as pf
import invert_main as invert
import mfr2D_matrix_gridtransform as grid_transf
import phantom_methods as pm
import factors_geometry_plots as fgp

import mfr2D_accessoires as mfr2D
import phantom_metrics as phm
import mfr_phantoms as mfrp
import phantom_comparison as phanc
import mfr_plot

import webapi_access as api
program = '20181010.032'
program_info, req = api.xpid_info(program=program)

t0 = program_info['programs'][0][
    'trigger']['1'][0]  # in ns
t4 = program_info['programs'][0][
    'trigger']['4'][0]  # in ns

In [ ]:
nPhi = 30  # toroidal slices
nFS = 20  # radial slices
nL = 75  # 150  # poloidal slices
vpF = 1.35  # blowup from LCFS
N = 4  # [2, 4]  # detector seperation, short X long side, 8
nN = N[0] * N[1] if isinstance(N, list) else N
interpolation = 'triang'  # 'square'  # detector split mode (triang)
magconf = 'EIM_000'  # magnetic configuration

strgrid = interpolation[0] + 'N' + str(nN) + '_' + \
          str(nPhi) + 'x' + str(nFS) + 'x' + str(nL) + '_' + str(vpF) 
confstr = magconf[:4] + 'beta' + magconf[-3:] + '_' + interpolation[0] + \
          'N' + str(nN) + '_' + str(nPhi) + 'x' + str(nFS) + \
          'x' + str(nL) + '_' + str(vpF)
print('\\\ strgrid ' + strgrid + ' confstr ' + confstr)

In [ ]:
importlib.reload(invert)

(mesh2D,           # 0
 mesh3D,           # 1
 raw_volume,       # 2
 old_volume,       # 3
 volume,           # 4
 line_sections2D,  # 5
 line_sections3D,  # 6
 emissivity2D,     # 7
 emissivity3D,     # 8
 factors,          # 9
 reff,             # 10
 pos_lofs,         # 11
 minor_radius,     # 12
 reff_LoS,         # 13
 geometry,         # 14
 interpolated,     # 15
 LoS,              # 16
 vmec_label )= invert.main(
    nPhi=30, nL=150, nFS=20, vpF=1.25, N=[4, 2], 
    VMID='KJM_027', interp_method='square',  # triang
    tilt_deg=1., error_scale=0.0001,  # 0.1 mm (0.5 mm)  
    add_camera=False,
    new_type='VBCm',  # 'HBCm'
    artificial_HBCm=False,
    random_error=False,
    centered=False,
    fix_LoS=False,
    symmetric=False,
    tilt=False,
    plot=False,
    debug=False)

In [ ]:
loc = '//share.ipp-hgw.mpg.de/documents/pih/Documents/git/bolometer_mfr/output/'
strids = next(os.walk(loc))[1]
strids = [s for s in strids if s != 'phantom']

In [ ]:
loc = '//share.ipp-hgw.mpg.de/documents/pih/Documents/git/bolometer_mfr/output/phantom/'
strids = next(os.walk(loc))[1]
strids = [s for s in strids if s != 'bak']

In [ ]:
full, files, strgrids, xpids = [], [], [], []

aniModes = []
kanis = []
reduceds = []
values = []
nigs = []
times = []
RGSs = []

nNs, nPhis, nLs, nFSs, vpFs, magconfs, interpolations = \
    [], [], [], [], [], [], []

for directory in strids:
    A = next(os.walk(loc + directory + '/'))[1]
    for a in A:
        if a != 'bak':
            B = next(os.walk(loc + directory + '/' + a + '/'))[2]
            for b in B:
                if 'mfr1D.dat' in b:
                    if 'no_ani' in b:
                        continue
                    if directory in ['20181010.30', '20181010.032',
                                     '20181010.036']:
                        magconfs.append('EIM_065')
                    if directory in ['20180725.044', '20180809.013',
                                     '20181011.012']:
                        magconfs.append('KJM_027')
                    # else:
                    #     continue
                    if directory in ['20181004.024']:
                        magconfs.append('DBM_024')
                    if directory in ['20180719.022','20180920.029',
                                       '20180920.031', '20180920.032',
                                       '20180920.049',
                                       '20181016.016', '20181016.023']:
                        magconfs.append('EIM_000')

                    # magconfs.append('EIM_000')

                    full.append(b)
                    if not 'phan' in b:
                        file = b.replace(directory, '').replace(
                            a, '').replace('mfr1D.dat', '')
                    else:
                        file = b.replace(directory, '').replace(
                            'phan_', '').replace('mfr1D.dat', '')

                    files.append(file)
                    strgrids.append(a)
                    xpids.append(directory)

                    if 'reduced' in file:
                        reduceds.append(True)
                    else:
                        reduceds.append(False)

                    if 'RGS' in file:
                        RGSs.append(True)
                        file = file.replace('RGS_', '')
                    else:
                        RGSs.append(False)

                    ani = int(file.split('aniM', 1)[1][0])
                    aniModes.append(ani)

                    c = file.replace('aniM' + str(ani) + '_', '')
                    kani = [float(x) for x in c.split('_')[:2]]
                    kanis.append(kani)

                    if ani == 3:
                        vals = int(file.split('nT', 1)[1].split('_')[0])
                    if ani == 4:
                        vals = [int(x) for x in file.split(
                                'nT', 1)[1].replace('nW', '').split('_')[:2]]
                    values.append(vals)

                    nig = int(file.split('nigs', 1)[1][0])
                    nigs.append(nig)

                    t = float(file.split('_')[-2])
                    times.append(t)

                    N = int(a[2:3])
                    nNs.append(N)

                    vpF = float(a.split('_')[-1])
                    vpFs.append(vpF)

                    nPhi, nFS, nL = [
                        int(x) for x in a.split('_')[1].split('x')]
                    nPhis.append(nPhi)
                    nLs.append(nL)
                    nFSs.append(nFS)


In [ ]:
tomos, unique_tomos = xpids, np.unique([s[:12] for s in np.unique(xpids)])

In [ ]:
phantoms, unique_phans = xpids, np.unique([s[:8] for s in np.unique(xpids)])
unique_phans

In [ ]:
importlib.reload(mfr2D)
importlib.reload(mfr_plot)
importlib.reload(phm)

for file, F, nL, nFS, strgrid, program, ani, kani, \
        vals, time, magconf, reduced, RGS in zip(
        full, files, nLs, nFSs, strgrids, xpids, aniModes, kanis,
        values, times, magconfs, reduceds, RGSs):
    print(nL, nFS, strgrid, program, ani,
          kani, vals, time, magconf, reduced, RGS, end=' ')

    try:
        # mfr_results = mfr2D.get_mfr_results(
        #     program=program, no_ani=ani, kani=kani, nVals=vals,
        #     nigs=1, times=time, grid_nt=nL, grid_nr=nFS, strgrid=strgrid,
        #     base='_mfr1D', magconf=magconf[:4] + 'beta' + magconf[-3:],
        #     new_type=None, add_camera=False, reduced=reduced,
        #     phantom=True, RGS=RGS, plot=False, saving=True, debug=False)

        phm_results = phm.compare_tomo_phantom(
            label=program, no_ani=ani, kani=kani, nVals=vals,
            nigs=1, times=time, nL=nL, nFS=nFS, strgrid=strgrid,
            base='_mfr1D', magconf=magconf[:4] + 'beta' + magconf[-3:],
            new_type=None, add_camera=False, reduced=reduced,
            RGS=RGS, plot=False, saving=True, debug=False)

        # print('success')
    except Exception as e:
       print('failed')
       pass

In [ ]:
importlib.reload(mfrp)
importlib.reload(mfr_plot)
mfrp.ch_pow_phantoms(
    strgrid=strgrid,
    sigma=0.2,  # smoothness in phantom, in (W/m^3)/m
    x0=[5., 0.],  # hot spot location if chosen, in m
    amplitude=1.e6,  # maximum, in W/m^3
    add_noise=False,  #  random error
    error=0.025,  # in % of max amplitude, W/m^3
    in_label='_EIM_beta000_addedARTv_sN4_30x20x75_1.3',
    save_base='../results/INVERSION/MFR/',
    new_type='ARTv',  # new artificial camera type 
    add_camera=True,  # artificial camera
    systematic_error=False,  # camera geometry error implemented
    debug=False)  # debugging

In [ ]:
importlib.reload(mfrp)
importlib.reload(mfr_plot)
mfrp.ch_pow_phantoms(
    strgrid=strgrid,
    sigma=0.4,  # smoothness in phantom, in (W/m^3)/m
    x0=[5., 0.],  # hot spot location if chosen, in m
    amplitude=1.e6,  # maximum, in W/m^3
    add_noise=False,  #  random error
    error=0.025,  # in % of max amplitude, W/m^3
    in_label='_' + confstr,
    save_base='../results/INVERSION/MFR/',
    new_type=None,  # new artificial camera type 
    add_camera=False,  # artificial camera
    systematic_error=False,  # camera geometry error implemented
    debug=False)  # debugging

In [ ]:
nPhi = 30
nFS = 20
nL = 75
vpF = 1.3
N = 4  # [2, 4]
nN = N[0] * N[1] if isinstance(N, list) else N
interpolation = 'square'  # 'square'
magconf = 'EIM_000'  # 'KJM_027'  # 'EIM_000'  # 'EIM_065'  # 'DBM_000'
strgrid = interpolation[0] + 'N' + str(nN) + '_' + \
          str(nPhi) + 'x' + str(nFS) + 'x' + str(nL) + '_' + str(vpF)

print('\\\ magconf:', magconf, '\n\\\ strgrid:', strgrid)

In [ ]:
label = 'Rprof_R0.85_sig10.4_sig20.05_mx1.0e+06'
ani, kani, vals = 4, [20., 40.], [15, 4]

importlib.reload(phm)
importlib.reload(mfr2D)
importlib.reload(mfr_plot)

for val in [1., 4., 6]:
    phantom_results = phm.compare_tomo_phantom(
        nFS=nFS, nL=nL, N=nN, no_ani=ani, kani=[20., val], nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='tN4_30x20x75_1.35', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
l1, l2 = 'dfs_R1.1_', '_mx1.0e+06'
ani, kani, vals = 3, [40., 40.], 15

importlib.reload(phm)
importlib.reload(mfr2D)
importlib.reload(mfr_plot)

for r2 in ['0.4', '0.5', '0.6', '0.7', '0.8', '0.85', '0.9', '0.95']:
    phantom_results = phm.compare_tomo_phantom(
        nFS=nFS, nL=nL, N=nN, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=l1 + r2 + l2,
        strgrid='tN4_30x20x75_1.35', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'asym_R1.1_m4_mx1.0e+06'
ani, kani, vals = 4, [40., 4.], [14, 2]

importlib.reload(phm)
importlib.reload(mfr2D)
importlib.reload(mfr_plot)

phantom_results = phm.compare_tomo_phantom(
    nFS=nFS, nL=nL, N=nN, no_ani=ani, kani=kani, nVals=vals,
    nigs=1, times=0.11,
    label=label,  # l1 + r2 + l2,
    strgrid='tN4_30x20x75_1.35', base='_mfr1D', vmec_ID=magconf,
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    add_camera=False, new_type='ARTm', reduced=False,
    plot=True, saving=True, debug=False)

In [ ]:
l1, l2 = 'sym_R1.1_m', '_mx1.0e+06'
ani, kani, vals = 4, [40., 4.], [14, 2]

importlib.reload(phm)
importlib.reload(mfr2D)
importlib.reload(mfr_plot)

for m in ['6', '7']:
    phantom_results = phm.compare_tomo_phantom(
        nFS=nFS, nL=nL, N=nN, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=l1 + m + l2,
        strgrid='tN4_30x20x75_1.35', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type='ARTm', reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
l1, l2 = 'sym_R1.1_m', '_mx1.0e+06_ARTm'
ani, kani, vals = 4, [20., 4.], [15, 2]

importlib.reload(phm)
importlib.reload(mfr2D)
importlib.reload(mfr_plot)

for m in ['5', '8']:
    phantom_results = phm.compare_tomo_phantom(
        nFS=nFS, nL=nL, N=nN, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=l1 + m + l2,
        strgrid=strgrid, base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=True, new_type='ARTm', reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
l1, l2 = 'sym_R1.1_m', '_mx1.0e+06_ARTv'
ani, kani, vals = 4, [20., 4.], [15, 2]

importlib.reload(phm)
importlib.reload(mfr2D)
importlib.reload(mfr_plot)

for m in ['5', '8']:
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=75, N=4, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=l1 + m + l2,
        strgrid='sN4_30x20x75_1.3', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=True, new_type='ARTv', reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
l1, l2 = 'ring_R', '_mx1.0e+06'
ani, kani, vals = 3, [10., 10.], 10

importlib.reload(phm)
importlib.reload(mfr_plot)

for r in ['0.3', '0.4', '0.5', '0.6', '0.7',
          '0.8', '0.9', '1.0', '1.1', '1.2', '1.3']:
    for rns in ['', '_rns5.0']:
        phantom_results = phm.compare_tomo_phantom(
            nFS=nFS, nL=nL, N=nN, no_ani=ani, kani=kani, nVals=vals,
            nigs=1, times=0.11,
            label=l1 + r + l2 + rns,
            strgrid=strgrid, base='_mfr1D', vmec_ID=magconf,
            magconf=magconf[:4] + 'beta' + magconf[-3:],
            add_camera=False, new_type='MIRh', reduced=False,
            plot=True, saving=True, debug=False)

In [ ]:
label = 'pos_mesh_x5.75_y0.0_mx1.0e+06'
ani, kani, vals = 4, [10., 10.], 10

importlib.reload(phm)
importlib.reload(mfr_plot)

for kani, vals in zip(
        [[1., 20.], [.5, 10.], [.5, 40.], [3., 40.], [1., 20.], [4., 20.]],
        [[14, 1], [15, 1], [16, 1], [16, 1], [16, 1], [14, 1]]):
    phantom_results = phm.compare_tomo_phantom(
        nFS=nFS, nL=75, N=nN, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='tN4_30x20x75_1.3', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'pos_mesh_x4.75_y0.0_mx1.0e+06'
ani, kani, vals = 4, [10., 10.], 10

importlib.reload(phm)
importlib.reload(mfr_plot)

for kani, vals in zip(
        [[2., 20.], [1., 20.]], [[14, 1], [14, 1]]):
    phantom_results = phm.compare_tomo_phantom(
        nFS=nFS, nL=75, N=nN, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='tN4_30x20x75_1.3', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
l1, l2 = 'pos_mesh_x', '.75_y0.0_mx1.0e+06'
ani, kani, vals = 4, [1., 20.], [14, 1]

importlib.reload(phm)
importlib.reload(mfr_plot)

for x in ['5', '4']:
    phantom_results = phm.compare_tomo_phantom(
        nFS=nFS, nL=75, N=nN, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=l1 + x + l2,
        strgrid='tN4_30x20x75_1.35', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'symR1.1_fsR0.8_mx1.0e+06'
ani, kani, vals = 4, [40., 4.], [15, 2]

importlib.reload(phm)
importlib.reload(mfr_plot)

phantom_results = phm.compare_tomo_phantom(
    nFS=nFS, nL=75, N=nN, no_ani=ani, kani=kani, nVals=vals,
    nigs=1, times=0.11,
    label=label,
    strgrid='tN4_30x20x75_1.35', base='_mfr1D', vmec_ID=magconf,
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    add_camera=False, new_type=None, reduced=False,
    plot=True, saving=True, debug=False)

In [ ]:
label = 'symR1.1_fsR0.8_mx1.0e+06'
ani, kani, vals = 4, [40., 4.], [15, 2]

importlib.reload(phm)
importlib.reload(mfr_plot)

for ani, kani, vals in zip(
        [3, 4, 4, 4],
        [[2., .1], [2., .1], [2., .3], [2., .35]],
        [15, [15, 2], [15, 2], [15, 2]]):
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='sN8_30x20x150_1.35', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'symR1.1_fsR0.8_mx1.0e+06'
ani, kani, vals = 4, [40., 4.], [15, 2]

importlib.reload(phm)
importlib.reload(mfr_plot)

for ani, kani, vals in zip(
        [4, 4], [[2., .6], [20., 3.]], [[15, 2], [15, 2]]):
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='sN8_30x20x150_1.3', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'symR0.8_fsR1.1_mx1.0e+06'
ani, kani, vals = 4, [40., 4.], [15, 2]

importlib.reload(phm)
importlib.reload(mfr_plot)

for ani, kani, vals in zip(
        [4, 4], [[5., 20.], [3., 20.]], [[15, 2], [15, 2]]):
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='sN8_30x20x150_1.3', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'blind_test_ones_mx1.0e+06'
ani, kani, vals = 3, [.1, 2.], 15

importlib.reload(phm)
importlib.reload(mfr_plot)

for ani, kani, vals in zip(
        [1, 3], [[1., 1.], [1., 1.]], [10, 10]):
    phantom_results = phm.compare_tomo_phantom(
        nFS=nFS, nL=75, N=nN, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='tN4_30x20x75_1.3', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'blind_test_ones_mx1.0e+06'
ani, kani, vals = 3, [1., 1.], 15

importlib.reload(phm)
importlib.reload(mfr_plot)

phantom_results = phm.compare_tomo_phantom(
    nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
    nigs=1, times=0.11,
    label=label,
    strgrid='sN8_30x20x150_1.35', base='_mfr1D', vmec_ID=magconf,
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    add_camera=False, new_type=None, reduced=False,
    plot=True, saving=True, debug=False)

In [ ]:
label = 'strahl_ID000'
ani, kani, vals = [3, 3], [[10., 10.], [2., 2.]], [10, 15]

importlib.reload(phm)
importlib.reload(mfr_plot)

for ID in ['91', '92', '93', '94']:
    for k, vp in enumerate(['1.3', '1.35']):
        phantom_results = phm.compare_tomo_phantom(
            nFS=20, nL=150, N=8, no_ani=ani[k], kani=kani[k], nVals=vals[k],
            nigs=1, times=0.11,
            label=label + ID,
            strgrid='sN8_30x20x150_' + vp, base='_mfr1D', vmec_ID=magconf,
            magconf=magconf[:4] + 'beta' + magconf[-3:],
            add_camera=False, new_type=None, reduced=False,
            plot=True, saving=True, debug=False)

In [ ]:
label = 'ring_R1.1_mx1.0e+06_tilt'
ani, kani, vals = 3, [2., 2.], 15

importlib.reload(phm)
importlib.reload(mfr_plot)

for ang in ['1.0', '-1.0']:
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label + ang,
        strgrid='sN8_30x20x150_1.35', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'sym_R1.1_m8_mx1.0e+06'
ani, kani, vals = 4, [40., 4.], [14, 2]

importlib.reload(phm)
importlib.reload(mfr_plot)

phantom_results = phm.compare_tomo_phantom(
    nFS=20, nL=75, N=4, no_ani=ani, kani=kani, nVals=vals,
    nigs=1, times=0.11,
    label=label,
    strgrid='tN4_30x20x75_1.35', base='_mfr1D', vmec_ID=magconf,
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    add_camera=False, new_type=None, reduced=False,
    plot=True, saving=True, debug=False)

In [ ]:
label = 'sym_R1.1_m8_mx1.0e+06'
ani, kani, vals = 4, [20., 4.], [15, 2]

importlib.reload(phm)
importlib.reload(mfr_plot)

phantom_results = phm.compare_tomo_phantom(
    nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
    nigs=1, times=0.11,
    label=label,
    strgrid='sN8_30x20x150_1.3', base='_mfr1D', vmec_ID=magconf,
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    add_camera=False, new_type=None, reduced=False,
    plot=True, saving=True, debug=False)

In [ ]:
label = 'sym_R1.1_m8_mx1.0e+06'
ani, kani, vals = 3, [2., .1], 15

importlib.reload(phm)
importlib.reload(mfr_plot)

phantom_results = phm.compare_tomo_phantom(
    nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
    nigs=1, times=0.11,
    label=label,
    strgrid='sN8_30x20x150_1.35', base='_mfr1D', vmec_ID=magconf,
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    add_camera=False, new_type=None, reduced=False,
    plot=True, saving=True, debug=False)

In [ ]:
label = 'sym_R1.1_m8_mx1.0e+06'
ani, kani, vals = 3, [2., .1], 15

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

for kani in [[2., .1], [2., .25], [2., .3]]:
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='sN8_30x20x150_1.35', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False, RGS=True,
        plot=True, saving=True, debug=False)

In [ ]:
l1, l2, l3 = 'symR', '_fsR', '_mx1.0e+06_ARTm'
ani, kani, vals = 3, [2., .1], 15

importlib.reload(phm)
importlib.reload(mfr2D)
importlib.reload(mfr_plot)

for r1, r2 in zip(['0.8', '1.1'], ['1.1', '0.8']):
    for kani in [[2., .1], [.1, 2.]]:
        phantom_results = phm.compare_tomo_phantom(
            nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
            nigs=1, times=0.11,
            label=l1 + r1 + l2 + r2 + l3,
            strgrid='sN8_30x20x150_1.3', base='_mfr1D', vmec_ID=magconf,
            magconf=magconf[:4] + 'beta' + magconf[-3:],
            add_camera=True, new_type='ARTm', reduced=False,
            plot=True, saving=True, debug=False)

In [ ]:
label = 'sym_R1.1_m5_mx1.0e+06'
ani, kani, vals = 4, [40., 4.], [14, 2]

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

for kani, vals in zip(
        [[40., 4.], [40., 4.], [40., 3.], [40., 3.], [40., 1.],
         [40., 5.], [40., 5.], [40., 5.]],
        [14, 15, 15, 14, 15, 14, 15, 16]):
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=75, N=4, no_ani=ani, kani=kani, nVals=[vals, 2],
        nigs=1, times=0.11,
        label=label,
        strgrid='tN4_30x20x75_1.35', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False, RGS=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'sym_R1.1_m5_mx1.0e+06'
ani, kani, vals = 3, [2., .1], 15

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

for strgrid, ani, kani, vals in zip(
        ['sN8_30x20x150_1.35', 'sN8_30x20x150_1.3'],
        [3, 4],
        [[2., .1], [20., 4.]],
        [15, [15, 2]]):
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid=strgrid, base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False, RGS=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'asym_fsR1.0_m1_mx1.0e+06'
ani, kani, vals = 3, [2., .1], 14

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

for kani in [[2., .5], [2., 2.]]:
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='sN8_30x20x150_1.3', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False, RGS=False,
        plot=True, saving=True, debug=False)

In [ ]:
label = 'ring_R1.0_mx1.0e+06'
ani, kani, vals = 3, [10.,10], 10

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

phantom_results = phm.compare_tomo_phantom(
    nFS=20, nL=75, N=4, no_ani=ani, kani=kani, nVals=vals,
    nigs=1, times=0.11,
    label=label,
    strgrid='tN4_30x20x75_1.3', base='_mfr1D', vmec_ID=magconf,
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    add_camera=False, new_type=None, reduced=False, RGS=False,
    plot=True, saving=True, debug=False)

In [ ]:
label = 'ring_R1.0_mx1.0e+06'
ani, kani, vals = 3, [2., .5], 14

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

phantom_results = phm.compare_tomo_phantom(
    nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
    nigs=1, times=0.11,
    label=label,
    strgrid='sN8_30x20x150_1.3', base='_mfr1D', vmec_ID=magconf,
    magconf=magconf[:4] + 'beta' + magconf[-3:],
    add_camera=False, new_type=None, reduced=False, RGS=False,
    plot=True, saving=True, debug=False)

In [ ]:
l1, l2 = 'asym_fsR1.0_m1_offset', '_mx11.0e+06_mx01.5x'
ani, kani, vals = 3, [.3, .3], 14
# ani, kani, vals = 3, [1.5, .25], 14

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

for offset in ['0', '90', '180', '270']:
    for kani in [[.3, .3], [1.5, .25]]:
        phantom_results = phm.compare_tomo_phantom(
            nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
            nigs=1, times=0.11,
            label=l1 + offset + l2,
            strgrid='sN8_30x20x150_1.3', base='_mfr1D', vmec_ID=magconf,
            magconf=magconf[:4] + 'beta' + magconf[-3:],
            add_camera=False, new_type=None, reduced=False, RGS=False,
            plot=True, saving=True, debug=False)

In [ ]:
label = 'dfs_0.7_1.0_asymR0_m1_mx1.0e+06'
ani, kani, vals = 3, [.3, .3], 14

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

for kani in [[.3, .3], [2., .5], [.5, 2.]]:
    phantom_results = phm.compare_tomo_phantom(
        nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
        nigs=1, times=0.11,
        label=label,
        strgrid='sN8_30x20x150_1.3', base='_mfr1D', vmec_ID=magconf,
        magconf=magconf[:4] + 'beta' + magconf[-3:],
        add_camera=False, new_type=None, reduced=False, RGS=False,
        plot=True, saving=True, debug=False)

In [ ]:
l1, l2, l3, l4 = 'asym_m1R0.7_fsR1.0_off', '_mx1_', 'e+06_mx0_', 'e+06'
ani, kani, vals = 3, [.3, .3], 14

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

for offset in ['0', '90', '180', '270']:
    for m1, m2 in zip(['2.0', '1.0'], ['1.0', '1.0']):
        for RGS in [True, False]:
            phantom_results = phm.compare_tomo_phantom(
                nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
                nigs=1, times=0.11,
                label=l1 + offset + l2 + m1 + l3 + m2 + l4,
                strgrid='sN8_30x20x15 0_1.3', base='_mfr1D', vmec_ID=magconf,
                magconf=magconf[:4] + 'beta' + magconf[-3:],
                add_camera=False, new_type=None, reduced=False, RGS=RGS,
                plot=True, saving=True, debug=False)

In [ ]:
label = 'dfs_0.7_1.0_asym_m1_mx1.0e+06'
ani, kani, vals = 3, [.3, .3], 14

importlib.reload(phm)
importlib.reload(mfr_plot)
importlib.reload(mfr2D)

for kani in [[2., .3], [.3, .3]]:
    for RGS in [True, False]:
        phantom_results = phm.compare_tomo_phantom(
            nFS=20, nL=150, N=8, no_ani=ani, kani=kani, nVals=vals,
            nigs=1, times=0.11,
            label=label,
            strgrid='sN8_30x20x150_1.3', base='_mfr1D', vmec_ID=magconf,
            magconf=magconf[:4] + 'beta' + magconf[-3:],
            add_camera=False, new_type=None, reduced=False, RGS=RGS,
            plot=True, saving=True, debug=False)

In [ ]:
loc = '//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/results/INVERSION/MFR/'

strgrids = next(os.walk(loc))[1]
strgrids.remove('backup')
strgrids.remove('sN2_10x10x10_1.3')
strgrids.remove('sN4_10x10x10_1.35')
strgrids.remove('tN2_10x10x10_1.35')

files, strings = [], []
for directory in strgrids:
    F = next(os.walk(loc + directory + '/'))[2]
    for f in F:
        if 'comp_phantom_tomogram' in f:
            files.append(f)
            strings.append(directory)

In [ ]:
%%script python --no-raise-error

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(files, strings):
    if "sym_R1.1_m5_mx1.0e+06_ani" in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='sym_R1.1_m5_kedge_scan',
    definition_space=[.1, .25, 1., 2., 3., 4., 4.5, 5., 5.5, 6.],
    abscissa_label='k$_{edge}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(files, strings):
    if "ring_R" in f and '_mx1.0e+06_aniM3_10.0_10.0' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='single_ring_radius_scan',
    definition_space=[.3, .4, .5, .6, .7, .8, .9, 1., 1.1, 1.2, 1.3],
    abscissa_label='radius [r$_{a}$]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(files, strings):
    if "ring_R" in f and '_mx1.0e+06_rns5.0_aniM3_10.0_10.0' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='single_ring_error5_radius_scan',
    definition_space=[.3, .4, .5, .6, .7, .8, .9, 1., 1.1, 1.2, 1.3],
    abscissa_label='radius [r$_{a}$]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(files, strings):
    if 'pos_mesh_x5.75_y0.0_mx1.0e+06' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='pos_mesh_x575_kcore_scan',
    definition_space=[.25, .5, 1., 1.5, 2., 3., 4.],
    abscissa_label='k$_core$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(files, strings):
    if 'pos_mesh_x4.75_y0.0_mx1.0e+06' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='pos_mesh_x475_kcore_scan',
    definition_space=[1., 1.5, 2.],
    abscissa_label='k$_core$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(files, strings):
    if 'dfs_R1.1_' in f and '_mx1.0e+06_aniM3_40.0_40.0' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='dfs_scan_inner_radius',
    definition_space=[.4, .5, .6, .7, .8, .85, .9, .95],
    abscissa_label='radius [r$_{a}$]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(files, strings):
    if 'asym_fsR1.0_m1_offset' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

# indices = (0, 1, 2, 7, 8, 3, 4, 5, 6)
indices = (0, 7, 3, 5)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='asym_m1_fs_polAngle_scan',
    # definition_space=[.0, .0, .0, 90., 90., 180., 180., 270., 270.],
    definition_space=[.0, 90., 180., 270.],
    abscissa_label='poloidal angle [°]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(files, strings):
    if 'asym_m1R0.7_fsR1.0_off' in f and 'mx1_1.0e+06_mx0_1' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

# indices = (0, 1, 6, 7, 2, 3, 4, 5)
indices = (0, 6, 2, 4)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='asym_m1_fs_mix_amp_polAngle_scan',
    # definition_space=[.0, .0, 90., 90., 180., 180., 270., 270.],
    definition_space=[.0, 90., 180., 270.],
    abscissa_label='poloidal angle [°]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(files, strings):
    if 'asym_m1R0.7_fsR1.0_off' in f and 'mx1_2.0e+06_mx0_1' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

# indices = (0, 1, 2, 3, 8, 9, 4, 5, 6, 7)
indices = (0, 8, 4, 6)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='asym_m1_fs_mix_amp_polAngle_scan',
    # definition_space=[.0, .0, .0, .0, 90., 90., 180., 180., 270., 270.],
    definition_space=[.0, 90., 180., 270.],
    abscissa_label='poloidal angle [°]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
loc = '//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/results/INVERSION/MFR/backup/'
strgrids = next(os.walk(loc))[1]

old_files, old_strings = [], []
for directory in strgrids:
    F = next(os.walk(loc + directory + '/'))[2]
    for f in F:
        if 'comp_phantom_tomogram' in f:
            old_files.append(f)
            old_strings.append(directory)
chopped = np.unique([x[22:65] for x in old_files])

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "dfs_R1.1" in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (0, 1, 2, 3, 5, 4, 7, 6)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='dfs_R1.1_inner_radius_scan_old',
    definition_space=[.4, .5, .6, .7, .8, .85, .9, .95],
    abscissa_label='radius [r$_{a}$]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "dfs_R" in f and 'R1.05_sig0.2' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (0, 1, 2, 4, 3, 5, 8, 7, 9)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='dfs_R1.05_inner_radius_scan_old',
    definition_space=[.4, .5, .6, .7, .75, .8, .9, .95, 1.],
    abscissa_label='radius [r$_{a}$]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "tomogram_ring_R" in f and '6_aniM3_1' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

# indices = (0, 1, 2, 4, 3, 5, 8, 7, 9)
# order, hits, locs = \
#     [order[i] for i in indices], \
#     [hits[i] for i in indices], \
#     [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='single_rign_radius_scan_old',
    definition_space=[.3, .4, .5, .6, .7, .8, .9, 1., 1.1, 1.2, 1.3],
    abscissa_label='radius [r$_{a}$]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "_fs_reff_ring_R0.8_" in f and '_0.8_nigs1' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 11)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='fs_reff_ring_kedge_scan_old',
    definition_space=[1.25, 1.3, 1.35, 1.4, 1.45, 1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8],
    abscissa_label='k$_{edge}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "_fs_reff_ring_R0.8_" in f and 'x0.5_kani1.5_' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (0, 2, 1, 4, 3, 6, 5, 7)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='fs_reff_ring_kcore_scan_old',
    definition_space=[.65, .7, .75, .8, .85, .9, .95, 1.],
    abscissa_label='k$_{core}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "pos_mesh_x5.75_y0.0" in f and '_0.8_nigs1' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='pos_mesh_x575_kedge_scan_old',
    definition_space=[1.25, 1.3, 1.35, 1.4, 1.45, 1.5, 1.55, 1.6, 1.65, 1.7, 1.75],
    abscissa_label='k$_{edge}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "pos_mesh_x5.75_y0.0" in f and '_kani1.5_' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='pos_mesh_x575_kcore_scan_old',
    definition_space=[.4, .45, .5, .55, .6, .65, .7, .75, .8, .85, .9, .95, 1.0, 1.05],
    abscissa_label='k$_{core}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "pos_mesh_x4.75_y0.0" in f and '_kani1.5_' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (1, 0, 3, 2, 5, 4, 7, 6, 8)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='pos_mesh_x475_kcore_scan_old',
    definition_space=[.6, .65, .7, .75, .8, .85, .9, .95, 1.],
    abscissa_label='k$_{core}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "_magax_sym_ring_R1.1_mode5_sig0.175_26x75_mx0.5_kani1.5_" in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (0, 2, 1, 4, 3, 6, 5, 8, 7, 9)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='magax_sym_ring_R1.1_mode5_kcore_scan_old',
    definition_space=[.55, .6, .65, .7, .75, .8, .85, .9, .95, 1.],
    abscissa_label='k$_{core}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "_magax_sym_ring_R1.1_mode5_sig0.175_26x75_mx0.5_kani" in f \
            and '0.8_nigs1_times0.11.json' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 11)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='magax_sym_ring_R1.1_mode5_kedge_scan_old',
    definition_space=[1.25, 1.3, 1.35, 1.4, 1.45, 1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8],
    abscissa_label='k$_{edge}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "_magax_fs_ring_R0.8_sig_0.175_26x75_mx0.5_kani1.5_" in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (0, 2, 1, 4, 3, 6, 5, 8, 7, 9)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='magax_fs_ring_R0.8_kcore_scan_old',
    definition_space=[.55, .6, .65, .7, .75, .8, .85, .9, .95, 1.],
    abscissa_label='k$_{core}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
order, hits, locs = [], [], []
for f, s in zip(old_files, old_strings):
    if "_magax_fs_ring_R0.8_sig_0.175_26x75_mx0.5_kani" in f and '_0.8_nigs1_' in f:
        order.append(f)
        hits.append([f])
        locs.append(s)
order, hits, locs = zip(*sorted(zip(order, hits, locs)))

indices = (0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 11)
order, hits, locs = \
    [order[i] for i in indices], \
    [hits[i] for i in indices], \
    [locs[i] for i in indices]

importlib.reload(phanc)
importlib.reload(mfr_plot)
results, objects = phanc.compare(
    label='magax_fs_ring_R0.8_kedge_scan_old',
    definition_space=[1.25, 1.3, 1.35, 1.4, 1.45, 1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8],
    abscissa_label='k$_{edge}$ [a.u.]',
    base_location='../results/INVERSION/MFR/',
    strgrid=locs,
    file_list=hits,
    saving=True,
    debug=False)

In [ ]:
importlib.reload(mfrp)
importlib.reload(mfr_plot)

mfrp.ch_pow_phantoms(
    strgrid=strgrid,
    sigma=0.2,  # smoothness in phantom, in (W/m^3)/m
    x0=[4.75, 0.],  # hot spot location if chosen, in m
    amplitude=1.e6,  # maximum, in W/m^3
    add_noise=False,  #  random error
    error=0.025,  # in % of max amplitude, W/m^3
    in_label='_' + confstr,
    save_base='../results/INVERSION/MFR/',
    new_type=None,  # new artificial camera type 
    add_camera=False,  # artificial camera
    systematic_error=False,  # camera geometry error implemented
    debug=False)  # debugging

In [ ]:
unique_tomos

In [ ]:
unique_phans

In [ ]:
importlib.reload(tm)
importlib.reload(mfrp)
importlib.reload(phm)

phantomData = tm.plot_statistics(group='comp_phantom_tomogram_strahl')
#                                , data=phantomData)

In [ ]:
importlib.reload(tm)
importlib.reload(mfrp)
importlib.reload(phm)

experimentData = tm.plot_statistics(group='xp_tomogram_tomo_20181011')
#                                   , data=experimentData)

In [ ]:
importlib.reload(tm)
importlib.reload(mfrp)

# (ecrh, pradh, pradv, wdia, dwdia, div, divertors) = 
data = tm.power_balance_tomogram(
    program='20181010.032', passVals=False, plot=True)

In [ ]:
importlib.reload(tm)
importlib.reload(mfrp)

# (ecrh, pradh, pradv, wdia, dwdia, div, divertors) = 
data = tm.power_balance_tomogram(
    program='20180725.044', passVals=False, plot=True)